In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv

from database_process import create_vector_store, add_documents
import processed_documents 

In [2]:
load_dotenv()

True

In [3]:
collection_name = "example_collection"
db_path = "./chroma_langchain_db" 
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

vector_store = create_vector_store(collection_name, db_path, passage_embeddings)

In [13]:
file_path = "data/Retrieval-Augmented Generation for.pdf"
texts = processed_documents.main(file_path=file_path)

전체 문서 개수: 155
헤더/푸터 제외 후 문서 개수: 136


In [14]:
texts

[Document(metadata={'id': 0, 'page': 1, 'category': 'heading1'}, page_content="<h1 id='0' style='font-size:20px'>Retrieval-Augmented Generation for<br>Knowledge-Intensive NLP Tasks</h1>"),
 Document(metadata={'id': 2, 'page': 1, 'category': 'paragraph'}, page_content="<br><p id='2' data-category='paragraph' style='font-size:16px'>Patrick Lewis†‡, Ethan Perez(cid:63),</p>"),
 Document(metadata={'id': 3, 'page': 1, 'category': 'paragraph'}, page_content="<p id='3' data-category='paragraph' style='font-size:16px'>Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,</p>"),
 Document(metadata={'id': 4, 'page': 1, 'category': 'paragraph'}, page_content="<p id='4' data-category='paragraph' style='font-size:16px'>Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†</p>"),
 Document(metadata={'id': 5, 'page': 1, 'category': 'paragraph'}, page_content="<p id='5' data-category='paragraph' style='font-size:14px'>†Facebook AI Research; 

In [15]:
vector_store = add_documents(vector_store, texts)

all_data = vector_store.get()
print(len(all_data['ids']))

210


In [16]:
from llm_process import generate_response
response = generate_response(vector_store, "Explain the Rag system")

[디버그] db : <langchain_chroma.vectorstores.Chroma object at 0x000002462CF25010>
[디버그] 0번째 시도 결과: {'question_summary': {'summary': 'RAG 시스템에 대해 설명해 주세요.'}, 'answer': {'main_content': ['RAG 시스템은 전통적인 시스템에 비해 몇 가지 중요한 이점을 제공합니다.', 'RAG 시스템은 복잡한 파이프라인 시스템과 도메인 특화 아키텍처를 필요로 하지 않으며, 중간 검색 감독 없이도 훈련될 수 있습니다.', 'RAG 시스템은 특정 상황에서 다른 시스템보다 성능이 뛰어납니다.'], 'metadata': [{'chunk_id': 38, 'page_number': 2, 'chunk_length': 1}, {'chunk_id': 70, 'page_number': 6, 'chunk_length': 1}, {'chunk_id': 70, 'page_number': 4, 'chunk_length': 1}]}, 'conclusion': {'conclusion': 'RAG 시스템은 복잡한 파이프라인 시스템과 도메인 특화 아키텍처 없이도 높은 성능을 발휘할 수 있는 시스템입니다. 이는 중간 검색 감독 없이도 훈련될 수 있으며, 특정 상황에서 다른 시스템보다 뛰어난 성능을 보입니다. 이러한 이점은 특히 RAG 시스템이 전통적인 시스템에 비해 더 나은 성능을 발휘할 수 있는 이유가 됩니다.'}}
[디버그] 0번째 시도 결과: {'next': False, 'score': 0.5, 'new_query': 'RAG 시스템이 무엇인지, 그리고 어떤 장점이 있는지 설명해 주세요.', 'reason': 'LLM의 답변은 RAG 시스템의 장점에 대해 언급하고 있지만, RAG 시스템이 무엇인지에 대한 구체적인 설명이 부족합니다. 문서에서는 RAG 시스템의 성능과 관련된 정보가 포함되어 있지만, RAG 시스템의 기본 개념이나 작동 방식에 대한 설명이 없습니다. 따라서,

In [18]:
response

({'question_summary': {'summary': 'RAG 시스템이 무엇인지, 그리고 RAG 시스템의 장점과 단점에 대해 설명해 주세요.'},
  'answer': {'main_content': ['RAG 시스템은 문서에서 정답을 추출하는 대신, 문서의 내용을 기반으로 새로운 답변을 생성할 수 있는 시스템입니다. 이는 문서에 정답이 명시적으로 존재하지 않더라도, 올바른 답변을 생성할 수 있는 능력을 가지고 있습니다.',
    'RAG 시스템의 장점 중 하나는, 문서에서 정답이 명시적으로 존재하지 않더라도 11.8%의 정확도로 올바른 답변을 생성할 수 있다는 점입니다. 이는 전통적인 추출 모델이 0%의 정확도를 보이는 상황에서도 RAG 시스템이 유리함을 보여줍니다.',
    'RAG 시스템의 단점으로는, 문서 검색 단계가 필요하다는 점이 있습니다. 이는 CAG 시스템과 비교했을 때, CAG 시스템이 검색 단계를 생략하여 더 빠르고 효율적이라는 점에서 RAG 시스템의 단점으로 작용할 수 있습니다.'],
   'metadata': [{'chunk_id': 63, 'page_number': 6, 'chunk_length': 2},
    {'chunk_id': 73, 'page_number': 4, 'chunk_length': 5}]},
  'conclusion': {'conclusion': 'RAG 시스템은 문서에서 직접적으로 답을 추출하는 것이 아니라, 문서의 내용을 바탕으로 새로운 답변을 생성할 수 있는 시스템입니다. 이는 문서에 명시적으로 답이 존재하지 않더라도, 높은 정확도로 올바른 답변을 생성할 수 있는 장점을 가지고 있습니다. 그러나, RAG 시스템은 문서 검색 단계가 필요하다는 점에서 CAG 시스템에 비해 효율성이 떨어질 수 있습니다. CAG 시스템은 검색 단계를 생략하여 더 빠르고 효율적인 처리가 가능하다는 점에서 RAG 시스템의 단점을 보완할 수 있습니다.'}},
 {'next': False,
  'score': 0.7,
  'new_